In [0]:
! pip3 install torch torchvision

Pytorch - n dimensional Tesor , can run on GPU ; Automatic differentiation 

Test Netwok - 

* simple fully connected Relu network  
* One hidden layer 
* trained with gradient descent to fit random data by maximizing the Euclidean distance between the network output and true output

* N - batch size
* D_in - input dimension
* H - hidden dimension
* D_out - output dimension

## Tensors 

### Numpy network

Before using Pytorch , creating a  network using numpy

Manually implementing forward and backward passes 

In [2]:
import numpy as np

# N - batch size , D_in - input dimension , 
# H - Hidden dimension , D_out - output dimension
N, D_in, H, D_out = 64,1000,100,10

# Creating random input and output data
x = np.random.randn(N,D_in)
y = np.random.randn(N,D_out)

# Randomly initializing weights
w1 = np.random.randn(D_in,H)
w2 = np.random.randn(H,D_out)

# learning rate
learning_rate =1e-6


# Training Loop
for t in range(500):
  # Forward Pass
  h = x.dot(w1)
  # relu squishes negetive value 
  h_relu = np.maximum(h,0)
  y_pred = h_relu.dot(w2)
  
  # Computing and printing loss
  loss = np.square(y_pred - y).sum()
  if t%50==0:
    print(t,loss)
  
  #Backprop
  # d(x^2)/dx = 2x 
  grad_y_pred = 2.0 * (y_pred - y)
  # h_relu.T - transpose of h_relu
  # gradient of w2
  grad_w2 = h_relu.T.dot(grad_y_pred)
  # gradoent o.f hidden relu
  grad_h_relu = grad_y_pred.dot(w2.T)
  # gradient of hidden unit
  grad_h = grad_h_relu.copy()
  # sqashing negetives
  grad_h[h<0]=0
  # gradient of w1
  grad_w1 = x.T.dot(grad_h)
  
  #Updating the weights
  w1 -= learning_rate*grad_w1
  w2 -= learning_rate*grad_w2
  
  if t==499:
    print('forward')
    print(f'[{x.shape}] * [{w1.shape}] = [{h.shape}] = max(val,0) = [{h_relu.shape}] * [{w2.shape}] =[{y_pred.shape}]')
    print('backward y\' -> w2\'(y\',h) -> h\'(y\',w2) -> w1\'(h\',x)')
    print(f'2 * ([{y_pred.shape}] - [{y.shape}]) = [{grad_y_pred.shape}]')
    print(f'[{h_relu.T.shape}] * [{grad_y_pred.shape}] = [{grad_w2.shape}]')
    print(f'[{grad_y_pred.shape}] * [{w2.T.shape}] = [{grad_h_relu.shape}]')
    print(f'[{grad_h_relu.shape}] = [{grad_h.shape}]')
    print(f'[{x.T.shape}] * [{grad_h.shape}] = [{grad_w1.shape}]')

0 30219360.47258518
50 18065.889810389395
100 1271.6621803559697
150 143.4248039489206
200 18.4409501783745
250 2.488475451111806
300 0.34359649442843065
350 0.04810594273538965
400 0.006795415647632441
450 0.000967698653036707
forward
[(64, 1000)] * [(1000, 100)] = [(64, 100)] = max(val,0) = [(64, 100)] * [(100, 10)] =[(64, 10)]
backward y' -> w2'(y',h) -> h'(y',w2) -> w1'(h',x)
2 * ([(64, 10)] - [(64, 10)]) = [(64, 10)]
[(100, 64)] * [(64, 10)] = [(100, 10)]
[(64, 10)] * [(10, 100)] = [(64, 100)]
[(64, 100)] = [(64, 100)]
[(1000, 64)] * [(64, 100)] = [(1000, 100)]


### Tensors Network

Tensors can be run on GPU and can keep track of computational graph and gradients 

Same network implemented using torch.tensors 

Manually implementing forward and backward pass 

In [3]:
import torch

dtype = torch.float
device = torch.device("cuda:0")

# N - batch size , D_in - input dimension 
# H - hidden state , D_out - output dimension

N, D_in, H, D_out = 64,1000,100,10

# Creating random input and output using tensors
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initializing weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate =1e-6

for t in range(500):
  # Forward Pass
  # mm - matrix multiply
  h = x.mm(w1)
  # removing negetives , clamp all elements b/w [min,max] and return resulting tensor
  h_relu = h.clamp(min=0)
  # predictions 
  y_pred = h_relu.mm(w2)
  
  # Computing and printing loss
  loss = (y_pred - y).pow(2).sum().item()
  if t%50==0:
    print(t,loss)
    
  # Backprop
  # y'
  grad_y_pred = 2.0 * (y_pred -y)
  # w2'
  grad_w2 = h_relu.t().mm(grad_y_pred)
  # h'
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  # squishing negetives
  grad_h[h<0]=0
  # w1'
  grad_w1 = x.t().mm(grad_h)
  
  # Updating weights
  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

0 25681834.0
50 16346.099609375
100 634.4251708984375
150 42.14482879638672
200 3.3945581912994385
250 0.3015325665473938
300 0.028702255338430405
350 0.0031278894748538733
400 0.0005423491238616407
450 0.00016499737103004009


## AutoGrad

In the previous networks we had to maually implement both forward and backward passes.

We can use **automatic differentiation **  to automatically compute thhe backward pass gradients in the network

Autograd uses ** Computational graph ** which is defined in the forward pass . Nodes in the graph will be tensors and edges will be functions that produce output tensor from input tensors . Backpropagating through this graph then allows for easy computation of gradients

Practice - Each tensor representes a node in a computational graph . If **x** is a tensor that has **x.requires_grad=True** , then **x.grad** is another tensor hplding the **gradient of tensor x w.r.t  some scalar value **

In [4]:
dtype = torch.float
device = torch.device("cuda:0")

# batch size , in,hidden,out params 
N, D_in, H, D_out = 64,1000,100,10

# Creating random input and outputs
# Setting requires_grad=False (default) indicates that gradients 
# need not be computed during backward pass
x = torch.randn(N,D_in,dtype=dtype,device=device)
y = torch.randn(N,D_out,dtype=dtype,device=device)

# Creating random weights 
# Setting requires_grad=True to ensure that gradients are calculated 
# w.r.t these tensors during backward pass
w1 = torch.randn(D_in, H, dtype=dtype,device=device,requires_grad=True)
w2 = torch.randn(H, D_out, dtype=dtype,device=device,requires_grad=True)

learning_rate =1e-6

for t in range(500):
  # Forward Pass
  # Same operations but as gradients are not being calculated manually
  # for backward pass , intermediate variables can be ignored 
  y_pred = x.mm(w1).clamp(min=0).mm(w2)
  
  # Computing and printing loss 
  # loss is a tensor of shape 1 , use item() to  get value
  loss = (y_pred - y).pow(2).sum()
  if t%50==0:
    print(t,loss.item())
  
  # Using autograd to calculate the gradients 
  # This call will compute the gradient of loss w.r.t all tensors with 
  # requires_grad=True 
  # After this call w1.grad, w2.grad will be tensors holding the 
  # gradient of the loss w.r.t w1 and w2 respectively
  loss.backward()
  
  # Manually updating weights using gradient descent. 
  # Wrapping in torch.no_grad() as weights have requires_grad=True but don't 
  # need to track this operation in autograd
  
  # Alternative way - operate on weight.data and weight.grad.data 
  # tensor.data gives a tensor that shares the storage with the the tensor
  # but does not track history
  # Also can use torch.optim.SGD to achieve tis 
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad
    
    # Manually setting gradients after updating weights
    w1.grad.zero_()
    w2.grad.zero_()
  
  

0 32710250.0
50 22505.185546875
100 1221.1851806640625
150 105.53775024414062
200 11.311941146850586
250 1.3562647104263306
300 0.17225566506385803
350 0.022683413699269295
400 0.003314564935863018
450 0.0006640014471486211


### Defining new Autograd Functions 

Each primitive autograd operator is just two functions that operate on tensors .

**Forward -** computes the output tensors from input tensors .

** Backward -** recieves the gradient of the output tensor w.r.t some scalar value and computes the gradient of the input tensor w.r.t  to the same scalar value 

Can easily define our own autograd operator by defining a subclass of **torch.autograd.Function** and implement **forward** and **backward** functions.
Can use the new autograd operator by constructing an instance and calling it , passing tensors containing input data.

Creating a custom Relu autograd function

In [0]:
class MyRelU(torch.autograd.Function):
  """
  Can implement our own custom autograd by 
  defining forward and backward passes
  which operate on tensors
  """
  
  @staticmethod
  def forward(ctx,input):
    """
    In the forward pass we recieve a tensor containing the input 
    and return a Tensor containing the output. ctx is a context 
    object that can be used to stash info for backward computation.
    Can save arbitrary objects for use in the backward pass using
    ctx.save_for_backward method
    """
    ctx.save_for_backward(input)
    return input.clamp(min=0)
  
  @staticmethod
  def backward(ctx,grad_output):
    """
    In the backward pass we recieve a tensor containing the gradient
    of loss w.r.t the output , and we need to compute the gradient
    of loss w.r.t input 
    """
    
    input, = ctx.saved_tensors
    grad_input = grad_output.clone()
    # setting the gradient =0 for the values of input
    # where value =0
    grad_input[input<0]=0
    return grad_input 

In [6]:
# Creating the same two layer network

dtype = torch.float
device = torch.device("cuda:0")

# Batch size , in,hidden, out params
N,D_in,H,D_out = 64,1000,100,10

# random input and outputs
x = torch.randn(N,D_in,dtype=dtype,device=device)
y = torch.randn(N,D_out,dtype=dtype,device=device)

# random weights 
w1 = torch.randn(D_in,H,dtype=dtype,device=device,requires_grad=True)
w2 = torch.randn(H,D_out,dtype=dtype,device=device,requires_grad=True)

learning_rate = 1e-6

for t in range(500):
  # To apply own function , using .apply method
  relu = MyRelU.apply
  
  # Forward Pass 
  y_pred = relu(x.mm(w1)).mm(w2)
  
  # Computing loss and print
  loss = (y_pred - y).pow(2).sum()
  if t%50==0:
    print(t,loss.item())
  
  # Using autograd for backward pass
  loss.backward()
  
  # updating weights using gradient descent
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad
    
    # Manually setting the gradients to zerro
    w1.grad.zero_()
    w2.grad.zero_()
    


0 31186850.0
50 17259.037109375
100 998.8756713867188
150 93.18627166748047
200 10.428625106811523
250 1.2847579717636108
300 0.16770946979522705
350 0.0229497030377388
400 0.003501452738419175
450 0.0007342651369981468


## TensorFlow : Static Graphs

Pytorch and tensorflow both define a computational graph and use
automatic differentition to compute gradients .

**Difference ** 
    
* Tensorflow the computational graphs are **static**
            
*  Pytorch computational graphs are **dynamic**

In Tensorflow we define the computational graph once and then execute the same graph over and over again , possibly feeding different inpuut data to the graph.

In Pytorch , each forward pass defines a new computational graph

**Static Graphs ** - advantage , can be optimized upfront ; framework meight decide to fuse some graph operations for efficiency or come up with a strategy for distributing the graph across many GPU's . 

If reusing the same graph over and over again => the potentially costly optimization can be amortized and same graph can be resused again and again.

** Control Flow **

some models we may wish to perform different computation for each data point.

Eg - RNN - unrolled for different number of timesteps for each data point which can be implemented in a loop . 

With **Static graph ** the loop construct needs to be a part of the graph. ( The reason Tensorflow provides **tf.scan** operator for embedding loops into the graph .)

With **Dynamic graphs** the situation is simpler: since we build the graph on-the-fly for each example , we can use normal imperative flow control to perform computation that differs for each input.

Using Tensorflow to fit the same network

In [2]:
import tensorflow as tf
import numpy as np


# First we set up the computational graph

# batch size , in , hidden , out params
N,D_in,H,D_out = 64,1000,100,10

# Create placeholders for the input  and output data 
# These will be filled with real data when the graph is executed

x = tf.placeholder(tf.float32,shape=(None,D_in))
y = tf.placeholder(tf.float32,shape=(None,D_out))

# Create variables for the weights and initialize them with random data
# A tensorflow variable persists its value across executions of graph
w1 = tf.Variable(tf.random_normal((D_in,H)))
w2 = tf.Variable(tf.random_normal((H,D_out)))

# Forward Pass 
# Compute the predicted y using operations on Tensorflow tesnors
# This code does not perform any numerical operations , it merely
# sets up the computational graph that will later execute
h = tf.matmul(x,w1)
h_relu = tf.maximum(h,tf.zeros(1))
y_pred = tf.matmul(h_relu,w2)

# Computing loass using operations on Tensorflow tensors
# Reduces `input_tensor` along the dimensions given in `axis`.
loss = tf.reduce_sum((y-y_pred)**2)

# Compute the gradient of loss w.r.t w1 and w2
# Constructs symbolic derivatives of sum of `ys` w.r.t. x in `xs`
grad_w1,grad_w2 = tf.gradients(loss, [w1,w2])

# Update weights using gradient descent
# To actually update the weights , we need to evaluate new_w1 and new_w2 
# when executing the graph 
# In Tensorflow the act of updating the value of weights is part of 
# the computational graph
# In Pyttorch this happens outside the computational graph
learning_rate =1e-6

new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

# Now the computational graph is built, enter a tensorflow session 
# to actually execute the graph
with tf.Session() as sess:
  # Run the graph once to initialize the weight variable w1,w2
  sess.run(tf.global_variables_initializer())
  
  # Create numpy arrays holding the actual data for x,y
  x_value = np.random.randn(N,D_in)
  y_value = np.random.randn(N,D_out)
  
  for t in range(500):
    # Execute the graph many times , each time it executes we want 
    # to bind x_value to x and y_value to y, specified with feed_dict
    # Each time we execute the graph we want to compute the values for
    # loss , new_w1,new_w2 ; The values of these tensors are returned
    # as numpy arrays
    loss_value,_,_ = sess.run([loss,new_w1,new_w2],
                             feed_dict={x:x_value,y:y_value})
    if t%50==0:
      print(t,loss_value)


0 32104396.0
50 12914.943
100 476.44772
150 27.931505
200 1.9787308
250 0.15744771
300 0.0137233585
350 0.0015263855
400 0.00030967587
450 0.00010701763


## nn.Module

### Pytorch:nn

Autograd and computational graphs can be too low level

When building of Neural Networks think of arranging the computation into **layers** ,
which have learnable parameters which will be optimized during learning

**nn** package defines a set of **Modules** whih is roughly equivalent to neural  network layer

A module recieves an input tensor and computes output tesnsor ,but may also hold internal state such as tensors containing learnable parameter 


In [7]:
# batch size , in,hidden,out params
N,D_in,H,D_out = 64,1000,100,10

# Create random input and output tensors
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

# Use the nn package to define outr model as a sequence of layers .
# nn.Sequential is a Module which cotains other modules , and 
# applies them in a sequence to produce its output.
# Each Linear Module computes output from input using a linear function
# and holds internal tensors for its weight and bias
model = torch.nn.Sequential(
        torch.nn.Linear(D_in,H),
        torch.nn.ReLU(),
        torch.nn.Linear(H,D_out))

# The nn package also contains definitions of popular loss functions 
# Using MSE as the loss function
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate=1e-4

for t in range(500):
  # Forward Pass 
  # compute predicted y by passing input x to the model.
  # Module object override the __call__ operator so they can be called 
  # like functions .
  # Pass a tensor as input and get a tensor of output data
  y_pred = model(x)
  
  # Compute and print loss 
  # Pass tensors containing predicted and true values of y
  # loss function returns tensor containing loss 
  loss = loss_fn(y_pred,y)
  if t%50==0:
    print(t,loss.item())
  
  # Manually setting the gradients as zero
  model.zero_grad()
  
  # Backward Pass 
  # Compute gradient od the loss w.r.t all the learnable parms
  # internally the parameters of each Module are stored in tensors 
  # with reuires_grad=True
  # Computing gradients for all learnable params
  loss.backward()
  
  # Update the weights using gradeint descent 
  with torch.no_grad():
    for param in model.parameters():
      param -= learning_rate*param.grad

0 679.7244873046875
50 27.946258544921875
100 1.8075411319732666
150 0.21464239060878754
200 0.03270195797085762
250 0.005752671975642443
300 0.0011209604563191533
350 0.00023694874835200608
400 5.314098598319106e-05
450 1.245086059498135e-05


### Pytorch:optim

So far updating weights manually using torch.no_grad() or .data  to avoid tracking history in autograd

Becomes complex for complex optimizers like AdaGrad, RMSProp, Adam

**optim** package in Pytorch abstracts the idea and provides implementation of commonly used optimization algos.

In [11]:
# batch size , in, hidden m out params
N, D_in, H, D_out =64, 1000, 100, 10

# Creating random x and y 
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

# Using nn Sequntial package to create the model
model = torch.nn.Sequential(
        torch.nn.Linear(D_in,H),
        torch.nn.ReLU(),
        torch.nn.Linear(H,D_out))

loss_fn = torch.nn.MSELoss(reduction="sum")

# Using the optim package to define an Optimizer that will update the weights
# Using Adam optimizer here - the first argument to Adam constructor tells the 
# optimizer which tenors to update
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

for t in range(500):
  # Forward Pass , computing predicted_y using x and model
  y_pred = model(x)
  
  # Computing and printing loss
  loss = loss_fn(y_pred,y)
  if t%50==0:
    print(t,loss.item())
    
  # Before backward pass , use optimizer to zero all gradients for the variables
  # it will update( which are the learnable weights of the model). This is because
  # by default gradients are accumulated in buffers (i.e. not overwritten) whenever
  # backwards is called . 
  optimizer.zero_grad()
  
  # Backward Pass , compute gradient of the loss w.r.t model params
  loss.backward()
  
  # Calling the step function on an Optimizer makes an update to its params
  optimizer.step()

0 714.4263916015625
50 223.8377227783203
100 55.706539154052734
150 8.764324188232422
200 1.0608502626419067
250 0.1430402547121048
300 0.017975587397813797
350 0.0017298394814133644
400 0.00013976407353766263
450 8.440280907962006e-06


### Pytorch:Custom nn Modules

Sometimes models can be more complex than a sequence of existing Modules, For these can define own Modules by subclassing **nn.Module** and defining   **forward** method - which recieves input tensors and produces output tensor using other modules or other autograd operations on tensors

In [0]:
class TwoLayerNN(torch.nn.Module):
  def __init__(self,D_in,H,D_out):
    """
    In constructor instantiating two nn.Linear modules and assigning them as 
    member variables
    """
    super(TwoLayerNN,self).__init__()
    
    self.l1 = torch.nn.Linear(D_in,H)
    self.l2 = torch.nn.Linear(H,D_out)
  
  def forward(self,x):
    """
    In forward function we accept a Tensor of input data and we must return a 
    Tensor of output data. We can use Modules defined in the constructor as well
    as arbitrary operators on Tensors
    """
    h_relu = self.l1(x).clamp(min=0)
    y_pred = self.l2(h_relu)
    return y_pred

In [12]:
# batch size , in ,hidden, out params
N,D_in,H,D_out = 64,1000,100,10

#Creating random x and y 
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

# Creating model
model = TwoLayerNN(D_in,H,D_out)

learning_rate =1e-4

# Creating a loss function and an optimizer 
# model.parameters() will contain the learnable parameters of the two
# nn.Linear modules which are memebers of the model
criterion = torch.nn.MSELoss(reduction="sum")
# passing the params to SGD optimizer
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

for t in range(500):
  # Forward Pass
  y_pred = model(x)
  
  # Calculating and printing loss
  loss = criterion(y_pred,y)
  if t%50==0:
    print(t,loss.item())
    
  # Zero gradients ,backward pass , update weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


0 629.7481689453125
50 28.894826889038086
100 1.7199313640594482
150 0.18932399153709412
200 0.02804916352033615
250 0.005026581697165966
300 0.0010181789984926581
350 0.00022453453857451677
400 5.2563187637133524e-05
450 1.2877923836640548e-05


### Pytorch:Control Flow + Weight Sharing

Creating a dynamic network - Fully connected ReLU network 
  
on each forward pass chooses a random number b/w 1 and 4 => that many hidden layers. Reusing the same weights multiple times to cimmpute the innermost hidden layers

Implement using loop and reusing the same Module multiple times when defining forward pass

In [0]:
import random

class DynamicNN(torch.nn.Module):
  def __init__(self,D_in,H,D_out):
    """
    In constructor defining the three nn.Linear instaances that will be 
    used in the forward pass
    """
    super(DynamicNN,self).__init__()
    self.input_layer = torch.nn.Linear(D_in,H)
    self.middle_layer = torch.nn.Linear(H,H)
    self.output_layer = torch.nn.Linear(H,D_out)
    
  def forward(self,x):
    """
    For forward pass , randomly choose either 0,1,2,3 and reuse the middle_layer
    module that many times to compute hidden layer computations
    
    Each Forward pass builds a dynamic computational graph , thus can use normal
    pythonic control statements -loop,if/else when defining forward pass of model
    """
    h_relu = self.input_layer(x).clamp(min=0)
    for _ in range(random.randint(0,3)):
      h_relu = self.middle_layer(h_relu).clamp(min=0)
    
    y_pred = self.output_layer(h_relu)
    return y_pred

In [16]:
# batch size , in , hidden , out params 
N,D_in,H,D_out = 64,1000,100,10

# Creating random x and y
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

# Creating the model
model = DynamicNN(D_in,H,D_out)

# Optimizer and loss function 
criterion = torch.nn.MSELoss(reduction="sum")
# Using momentum here as training can be slow
optimizer = torch.optim.SGD(model.parameters(),lr=1e-4,momentum=0.9)

for t in range(500):
  # Forward Pass 
  y_pred = model(x)
  
  # Computing and printing loss
  loss = criterion(y_pred,y)
  if t%50==0:
    print(t,loss.item())
    
  # Zero gradients , backward pass , update weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

0 656.40625
50 159.1815643310547
100 22.87323570251465
150 5.997773170471191
200 0.7470123767852783
250 1.5068737268447876
300 2.8965821266174316
350 0.7964998483657837
400 0.4362240135669708
450 5.319315433502197
